# Machine Learning with `scikit-learn`

## <img src='imgs/robotguy.png' alt="Smiley face" width="42" height="42" align="left">Learning Objectives
* * *
* Gain some high level knowledge around machine learning with a gentle/brief introduction
* Learn the importance of pre-processing data and how `scikit-learn` expects data
* See data transformations for machine learning in action
* Get an idea of your options for learning on training sets and applying model for prediction
* See what sort of metrics are commonly used in scikit-learn
* Learn options for model evaluation
* Become familiar with ways to make this process robust and simplified (pipelining and tuning parameters)

### For workshop (reorg)

Sections:
1. ML 101 w/ code examples (incl. taste of sklearn w/ logistic regression + accuracy scores and user entered sepal and petal measurements); intro to sklearn's approach
2. Our data: iris
3. Supervised
4. Unsupervised
* Evaluating a model
* What next (GridSearch & Pipeline)

Flow:
* a learner and visual (logistic regression, pairplot, accuracy, give some sepal and petal measurements)
  * what category of ML is logistic regression? (regression or classification?)
  * QUESTION: what should we have done first? Any ideas?  (EDA, visualize, pre-process if need be)
* flower pics
* peek at data
* preprocessing from sklearn
* Supervised - decision tree in detail, random forest
* Unsupervised - novelty detection aka anomoly detection (note that PCA & dimensionality reduction is unsupervised)
* Evaluate - metrics
* what do you do with this model?  what next?
* note: parameter tuning can be automated with GridSearch
* note: can test many algorithms at once with Pipeline

### some questions inline
Does this parameter when increased cause overfitting or underfitting?  what are the implications of those cases?<br><br>
Is it better to have too many false positives or too many false negatives?<br><br>
What is the diffrence between outlier detection and anomaly detection?

## Machine Learning 101

It's said in different ways, but I like the way Jake VanderPlas defines ML:

> Machine Learning is about building <b>programs with tunable parameters</b> (typically an array of floating point values) that are adjusted automatically so as to improve their behavior by <b>adapting to previously seen data</b>.

He goes on to say:

> Machine Learning can be considered a <b>subfield of Artificial Intelligence</b> since those algorithms can be seen as building blocks to make computers learn to behave more intelligently by somehow <b>generalizing</b> rather that just storing and retrieving data items like a database system would do.

(more [here](http://www.astroml.org/sklearn_tutorial/general_concepts.html))

ML is much more than writing a program.  ML experts write clever and robust algorithms which can generalize to answer different, but specific questions.  There are still types of questions that a certain algorithm can not or should not be used to answer.  I say answer instead of solve, because even with an answer one should evaluate whether it is a good answer or bad answer.  Also, just an in statistics, one needs to be careful about assumptions and limitations of an algorithm and the subsequent model that is built from it.  Here's my hand-drawn diagram of the machine learning process.<br>

<img src='imgs/ml_process.png' alt="Smiley face" width="550">
<br><br>


### Examples
Below, we are going to show a simple case of <i>classification</i>.  In the figure we show a collection of 2D data, colored by their class labels (imagine one class is labeled "red" and the other "blue").  

> The `fig_code` module is credited to Jake VanderPlas and was cloned from his github repo [here](https://github.com/jakevdp/sklearn_pycon2015) - also on our repo is his license file since he asked us to include that if we use his source code. :)

In [ ]:
# Plot settings for notebook

# so that plots show up in notebook
%matplotlib inline

# seaborn here is used for aesthetics.
# here, setting seaborn plot defaults (this can be safely commented out)
import seaborn; seaborn.set()

In [ ]:
# Import an example plot from the figures directory
from fig_code import plot_sgd_separator
plot_sgd_separator()

Above is the vector which best separates the two classes, "red" and "blue" using a classification algorithm called Stochastic Gradient Decent (don't worry about the detail yet).  The confidence intervals are shown as dashed lines. - FACT CHECK CI LINE COMMENT PLEASE

This demonstrates a very important aspect of ML and that is the algorithm is <i>generalizable</i>, i.e., if we add some new data, a new point, the algorithm can <i>predict</i> whether is should be in the "red" or "blue" category.


> <b>ML TIP:  ML can only answer 5 questions:</b>
* How much/how many?
* Which category?
* Which group?
* Is it weird?
* Which action?

As far as algorithms for learning a model (i.e. running some training data through an algorithm), it's nice to think of them in two different ways (with the help of the [machine learning wikipedia article](https://en.wikipedia.org/wiki/Machine_learning)).  The first way of thinking about ML, is by the type of information or <i>input</i> given to a system.  So, given that criteria there are three classical categories:
1.  Supervised learning - we get the data and the labels
2.  Unsupervised learning - only get the data (no labels)
3.  Reinforcement learning - reward/penalty based information (feedback)

Another way of categorizing ML approaches, is to think of the desired <i>output</i>:
1.  Classification
2.  Regression
3.  Clustering
4.  Density estimation
5.  Dimensionality reduction

--> This second approach (by desired <i>output</i>) is how `sklearn` categorizes it's ML algorithms.


#### The problem solved in supervised learning (e.g. classification, regression)

Supervised learning consists in learning the link between two datasets: the observed data X and an external variable y that we are trying to predict, usually called “target” or “labels”. Most often, y is a 1D array of length n_samples.

All supervised estimators in `sklearn` implement a `fit(X, y)` method to fit the model and a `predict(X)` method that, given unlabeled observations X, returns the predicted labels y.

Common algorithms you will use to train a model and then use trying to predict the labels of unknown observations are: <b>classification</b> and <b>regression</b>.  There are many types of classification and regression (for examples check out the `sklearn` algorithm cheatsheet below).

#### The problem solved in <i>un</i>supervised learning 
In machine learning, the problem of unsupervised learning is that of trying to find <b>hidden structure</b> in unlabeled data.

Unsupervised models have a `fit()`, `transform()` and/or `fit_transform()` in `sklearn`.

#### There are some instances where ML is just not needed or appropriate for solving a problem.
Some examples are pattern matching (e.g. regex), group-by and data mining in general (discovery vs. prediction).

#### EXERCISE: Should I use ML or can I get away with something else?

Looking back at previous years, by what percent did housing prices increase over each decade?<br>
Looking back at previous years, and given the relationship between housing prices and mean income in my area, given my income how much will a house be in two years in my area?<br>
A vacuum like roomba has to make a decision to vacuum the living room again or return to its base.<br>
Is this image a cat or dog?<br>
Are orange tabby cats more common than other breeds in Austin, Texas?<br>
Using my SQL database on housing prices, group my housing prices by whether or not the house is under 10 miles from a school.<br>
What is the weather going to be like tomorrow?<br>
What is the purpose of life?

## A very brief introduction to scikit-learn (aka `sklearn`)

This module is not meant to be a comprehensive introduction to ML, but rather an introduction to the current de facto tool for ML in python.  As a gentle intro, it is helpful to think of the `sklearn` approach having layers of abstraction.  This famous quote certainly applies:

> Easy reading is damn hard writing, and vice versa. <br>
--Nathaniel Hawthorne

In `sklearn`, you'll find you have a common programming choice: to do things very explicitly, e.g. pre-process data one step at a time, perhaps do a transformation like PCA, split data into traning and test sets, define a classifier or learner with desired parameterss, train the classifier, use the classifier to predict on a test set and then analyze how good it did.  

A different approach and something `sklearn` offers is to combine some or all of the steps above into a pipeline so to speak.  For instance, one could define a pipeline which does all of these steps at one time and perhaps even pits mutlple learners against one another or does some parameter tuning with a grid search (examples will be shown towards the end).  This is what is meant here by layers of abstraction.

So, in this particular module, for the most part, we will try to be explicit regarding our process and give some useful tips on options for a more automated or pipelined approach.  Just note, once you've mastered the explicit approaches you might want to explore `sklearn`'s `GridSearchCV` and `Pipeline` classes.

Here is `sklearn`'s algorithm diagram - (note, this is not an exhaustive list of model options offered in `sklearn`, but serves as a good algorithm guide).
![](imgs/ml_map.png)

### Your first model - a multiclass logistic regression on the iris dataset
* `sklearn` comes with this dataset ready-to-go for `sklearn`'s algorithms

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

# Leave one value out from training set - that will be test later on
X_train, y_train = iris.data[:-1,:], iris.target[:-1]

In [ ]:
from sklearn.linear_model import LogisticRegression

# our model - a multiclass regression
logistic = LogisticRegression()

# train on iris training set
logistic.fit(X_train, y_train)

X_test = iris.data[-1,:].reshape(1, -1)

y_predict = logistic.predict(X_test)

print('Predicted class %s, real class %s' % (
 y_predict, iris.target[-1]))

print('Probabilities of membership in each class: %s' % 
      logistic.predict_proba(X_test))

QUESTION:
* What would have been good to do before plunging right in to a logistic regression model?

#### Some terms you will encouter as a Machine Learnest

Term  | Definition
------------- | -------------
Training set  | set of data used to learn a model
Test set  | set of data used to test a model
Feature  | a variable (continuous, discrete, categorical, etc.) aka column
Target  | Label (associated with dependent variable, what we predict)
Learner  | Model or algorithm
Fit, Train | learn a model with an ML algorithm using a training set
Predict | w/ supervised learning, give a label to an unknown datum(data), w/ unsupervised decide if new data is weird, in which group, or what to do next with the new data
Accuracy | percentage of correct predictions ((TP + TN) / total)
Precision | percentage of correct positive predictions (TP / (FP + TP))
Recall | percentage of positive cases caught (TP / (FN + TP))

> PRO TIP: Are you a statitician?  Want to talk like a machine learning expert?  Here you go (from the friendly people at SAS ([here](http://www.sas.com/it_it/insights/analytics/machine-learning.html))): 

A Statistician Would Say  | A Machine Learnest Would Say
------------- | -------------
dependent variable  | target
variable  | feature
transformation  | feature creation
